<a href="https://colab.research.google.com/github/mag381/mukonoso/blob/main/e_Stat_API__02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

e-StatのAPIを使ってデータを取得しCSVファイルに保存する方法 <p>
https://qiita.com/Sapolas/items/32102561f6d23cd745ec

In [ ]:
!pandas==1.0.5
!requests==2.24.0
!tqdm==4.47.0
!urllib3==1.25.9
!xlrd==1.2.0
!xlwt==1.3.0

In [4]:
import sys
import urllib
import requests

In [2]:
import csv
import json
import xlrd
import zipfile
import functools
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [5]:
class EstatRestApiClient:
    """
    This is a simple python module class for e-Stat API (ver.3.0).
    See more details at https://www.e-stat.go.jp/api/api-info/e-stat-manual3-0
    """

    def __init__(self, api_version=None, app_id=None):
        self.base_url = "https://api.e-stat.go.jp/rest"
        self.api_version = "3.0" if api_version is None else api_version
        self.app_id = "65a9e884e72959615c2c7c293ebfaeaebffb6030" if app_id is None else app_id

    def _request_get(self, endpoint, logging=True, stream=True, **params):
        try:
            res = requests.get(endpoint, params=params, stream=stream)
            res.encoding = res.apparent_encoding
            if logging:
                print(res, "HTTP GET:", res.url)
        except requests.exceptions.RequestException as error:
            print(error)
            sys.exit(1)
        return res

    def getStatsList(self, format="csv", **kwargs):
        """
        2.1 統計表情報取得 (HTTP GET)

        Keyword Args:
        =============
        appId: str
            Application ID. *REQUIRED
        lang: str
            language ("J" or "E").
        surveyYears: str
            YYYY or YYYYMM or YYYYMM-YYYYMM
        statsField: str
            数値2桁：統計大分類で検索 or
            数値4桁：統計小分類で検索
        statsCode: str
            数値5桁：作成機関で検索 or
            数値8桁：政府統計コードで検索
        searchWord: str
            検索キーワード
        searchKind: 1,2,3
            1：統計情報(省略値)
            2：小地域・地域メッシュ
            3：社会・人口統計体系
        statsNameList: str
            Y：統計調査名一覧
        startPosition: int (default: 1)
            データの取得開始位置（1から始まる行番号）
            統計データを複数回に分けて取得する場合等、
            前回受信したデータの<NEXT_KEY>タグの値を指定する
        limit: int (default: 100000)
            データの取得行数
        updatedDate: str
            YYYY or YYYYMM or YYYYMMDD or YYYYMMDD-YYYYMMDD
        callback:
            コールバック関数
            only needed for `jsonp` format requests
        """
        params = kwargs
        params["appId"] = self.app_id if not "appId" in params else kwargs["appId"]

        if format == "xml":
            endpoint = f"{self.base_url}/{self.api_version}/app/getStatsList"
            res = self._request_get(endpoint, **params)
            # TODO: Refine & Test followings
            return res.text
        elif format == "json":
            endpoint = f"{self.base_url}/{self.api_version}/app/json/getStatsList"
            res = self._request_get(endpoint, **params)
            return res.json()
        elif format == "jsonp":
            endpoint = f"{self.base_url}/{self.api_version}/app/jsonp/getStatsList"
            res = self._request_get(endpoint, **params)
            # TODO: Refine & Test followings
            return res.text
        elif format == "csv":
            endpoint = f"{self.base_url}/{self.api_version}/app/getSimpleStatsList"
            res = self._request_get(endpoint, **params)
            return res.content.decode("utf-8")

    def getMetaInfoURL(self, params_dict, format="csv", **kwargs):
        """
        2.2 メタ情報取得 (HTTP GET)
        """
        params = kwargs
        params["appId"] = self.app_id if not "appId" in params else kwargs["appId"]

        if format == "xml":
            endpoint = f"{self.base_url}/{self.api_version}/app/getMetaInfo"
            res = self._request_get(endpoint, **params)
            # TODO: Refine & Test followings
            return res.text
        elif format == "json":
            endpoint = f"{self.base_url}/{self.api_version}/app/json/getMetaInfo"
            res = self._request_get(endpoint, **params)
            return res.json()
        elif format == "jsonp":
            endpoint = f"{self.base_url}/{self.api_version}/app/jsonp/getMetaInfo"
            res = self._request_get(endpoint, **params)
            # TODO: Refine & Test followings
            return res.text
        elif format == "csv":
            endpoint = f"{self.base_url}/{self.api_version}/app/getSimpleMetaInfo"
            res = self._request_get(endpoint, **params)
            return res.content.decode("utf-8")

    def getStatsData(self, params_dict, format="csv", **kwargs):
        """
        2.3 統計データ取得 (HTTP GET)

        Keyword Args:
        appId: str
            Application ID. *REQUIRED
        lang: str
            language ("J" or "E").
        dataSetId: str
            「データセット登録」で登録したデータセットID
        statsDataId: str
            統計表ID
        lvTab:
            絞り込み条件 > 表章事項 > 階層レベル
            "X" or "X-X" or "-X" or "X-"
        cdTab:
            絞り込み条件 > 表章事項 > 単一コード
            各メタ情報の項目コード
            "X1, X2, X3, ... XN"
        cdTabFrom
            絞り込み条件 > 表章事項 > コードFrom
            絞り込む範囲にある開始位置の項目コード
            "X"
        cdTabTo
            絞り込み条件 > 表章事項 > コードTo
            絞り込む範囲にある終了位置の項目コード
            "X"
        lvTime
            絞り込み条件 > 時間軸事項 > 階層レベル
            "X" or "X-X" or "-X" or "X-"
        cdTime
            絞り込み条件 > 時間軸事項 > 単一コード
        cdTimeFrom
            絞り込み条件 > 時間軸事項 > コードFrom
        cdTimeTo
            絞り込み条件 > 時間軸事項 > コードTo
        lvArea
            絞り込み条件 > 地域事項 > 階層レベル
        cdArea
            絞り込み条件 > 地域事項 > 単一コード
        cdAreaFrom
            絞り込み条件 > 地域事項 > コードFrom
        cdAreaTo
            絞り込み条件 > 地域事項 > コードTo
        lvCat01
            絞り込み条件 > 分類事項 > 階層レベル
        cdCat01
            絞り込み条件 > 分類事項 > 単一コード
        cdCat01From
            絞り込み条件 > 分類事項 > コードFrom
        cdCat01To
            絞り込み条件 > 分類事項 > コードTo
        startPosition: int (default: 1)
            データの取得開始位置（1から始まる行番号）
            統計データを複数回に分けて取得する場合等、
            前回受信したデータの<NEXT_KEY>タグの値を指定する
        limit: int (default: 100000)
            データの取得行数
        metaGetFlg: True or False
            メタ情報有無
        cntGetFlg: True or False
            件数取得フラグ
        sectionHeaderFlg : 1 or 2
            せくチョンヘッダフラグ
            - 1 : セクションヘッダを出力する (省略値)
            - 2 : セクションヘッダを取得しない
        callback:
            コールバック関数
            only needed for `jsonp` format requests


        =============

        """
        params = kwargs
        params["appId"] = self.app_id if not "appId" in params else kwargs["appId"]

        if format == "xml":
            endpoint = f"{self.base_url}/{self.api_version}/app/getStatsData"
            res = self._request_get(endpoint, **params)
            # TODO: Refine & Test followings
            return res.text
        elif format == "json":
            endpoint = f"{self.base_url}/{self.api_version}/app/json/getStatsData"
            res = self._request_get(endpoint, **params)
            return res.json()
        elif format == "jsonp":
            endpoint = f"{self.base_url}/{self.api_version}/app/jsonp/getStatsData"
            res = self._request_get(endpoint, **params)
            # TODO: Refine & Test followings
            return res.text
        elif format == "csv":
            endpoint = f"{self.base_url}/{self.api_version}/app/getSimpleStatsData"
            res = self._request_get(endpoint, **params)
            return res.content.decode("utf-8")

    def postDataset(self):
        """
        2.4 データセット登録 (HTTP POST)
        """
        endpoint = f"{self.base_url}/{self.api_version}/app/postDataset"
        pass

    def refDataset(self, format="xml", **kwargs):
        """
        2.5 データセット参照 (HTTP GET)
        """
        params = kwargs
        params["appId"] = self.app_id if not "appId" in params else kwargs["appId"]

        if format == "xml":
            endpoint = f"{self.base_url}/{self.api_version}/app/refDataset"
            pass
        elif format == "json":
            endpoint = f"{self.base_url}/{self.api_version}/app/json/refDataset"
            pass
        elif format == "jsonp":
            endpoint = f"{self.base_url}/{self.api_version}/app/jsonp/refDataset"
            pass

    def getDataCatalog(self, format="xml", **kwargs):
        """
        2.6 データカタログ情報取得 (HTTP GET)
        """
        params = kwargs
        params["appId"] = self.app_id if not "appId" in params else kwargs["appId"]

        if format == "xml":
            endpoint = f"{self.base_url}/{self.api_version}/app/getDataCatalog"
            pass
        elif format == "json":
            endpoint = f"{self.base_url}/{self.api_version}/app/json/getDataCatalog"
            pass
        elif format == "jsonp":
            endpoint = f"{self.base_url}/{self.api_version}/app/jsonp/getDataCatalog"
            pass

    def getStatsDatas(self, format="xml", **kwargs):
        """
        2.7 統計データ一括取得 (HTTP GET)
        """
        params = kwargs
        params["appId"] = self.app_id if not "appId" in params else kwargs["appId"]

        if format == "xml":
            endpoint = f"{self.base_url}/{self.api_version}/app/getStatsDatas"
            pass
        elif format == "json":
            endpoint = f"{self.base_url}/{self.api_version}/app/json/getStatsDatas"
            pass
        elif format == "csv":
            endpoint = f"{self.base_url}/{self.api_version}/app/getSimpleStatsDatas"
            pass

In [6]:
def get_json(url):
    """
    Request a HTTP GET method to the given url (for REST API)
    and return its response as the dict object.

    Args:
    ====
    url: string
        valid url for REST API
    """
    try:
        print("HTTP GET", url)
        r = requests.get(url)
        json_dict = r.json()
        return json_dict
    except requests.exceptions.RequestException as error:
        print(error)


def download_json(url, filepath):
    """
    Request a HTTP GET method to the given url (for REST API)
    and save its response as the json file.

    Args:
    url: string
        valid url for REST API
    filepath: string
        valid path to the destination file
    """
    try:
        print("HTTP GET", url)
        r = requests.get(url)
        json_dict = r.json()
        json_str = json.dumps(json_dict, indent=2, ensure_ascii=False)
        with open(filepath, "w") as f:
            f.write(json_str)
    except requests.exceptions.RequestException as error:
        print(error)


def download_str(url, filepath, enc="utf-8", dec="utf-8", logging=False):
    """
    Request a HTTP GET method to the given url (for REST API)
    and save its response as the local text file.

    Args:
    =====
    url: string
        valid url for REST API
    filepathe: string
        valid path to the destination file
    enc: string
        encoding type for a content in a given url
    dec: string
        decoding type for a content in a downloaded file
            dec = 'utf-8' for general env
            dec = 'sjis'  for Excel on Win
            dec = 'cp932' for Excel with extended JP str on Win
    logging: bool
        flag to display HTTP request status
    """
    try:
        r = requests.get(url, stream=True)
        if logging:
            print("HTTP GET",  f"[{r.status_code}]", url)
        with open(filepath, 'w', encoding=enc) as f:
            f.write(r.content.decode(dec))
    except requests.exceptions.RequestException as error:
        print(error)


def download_all_str(
        urls,
        filepathes,
        max_workers=10,
        enc="utf-8",
        dec="utf-8"):
    """
    Request some HTTP GET methods to the given urls (for REST API)
    and save each response as the local text file.
    (!! This method uses multi threading when calling HTTP GET requests
    and downloading files in order to improve the processing speed.)

    Args:
    =====
    urls: list of strings
        valid urls for REST API
    filepathes: list of strings
        valid pathes to the destination file
    max_workers: int
        max number of working threads of CPUs within executing this method.
    enc: string
        encoding type for a content in a given url
    dec: string
        decoding type for a content in a downloaded file
            dec = 'utf-8' for general env
            dec = 'sjis'  for Excel on Win
            dec = 'cp932' for Excel with extended JP str on Win
    logging: True/False
    """
    func = functools.partial(download_str, enc=enc, dec=dec)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(
            tqdm(executor.map(func, urls, filepathes), total=len(urls))
        )
        del results
        return


def download_bin(url, filepath, logging=False):
    """
    Request some HTTP GET methods to the given urls (for REST API)
    and save each response as the xls file.

    Args:
    =====
    urls: list of strings
        valid urls for REST API
    filepathes: list of strings
        valid pathes to the destination file
    logging: bool
        flag to display HTTP request status
    """
    try:
        r = requests.get(url, stream=True)
        if logging:
            print("HTTP GET",  f"[{r.status_code}]", url)
        with open(filepath, 'wb') as f:
            f.write(r.content)
    except requests.exceptions.RequestException as error:
        print(error)


def download_all_bin(urls, filepathes, max_workers=5):
    """
    Request some HTTP GET methods to the given urls (for REST API)
    and save each response as the local file.
    (!! This method uses multi threading when calling HTTP GET requests
    and downloading files in order to improve the processing speed.)

    Args:
    =====
    urls: list of strings
        valid urls for REST API
    filepathes: list of strings
        valid pathes to the destination file
    max_workers: int
        max number of working threads of CPUs within executing this method.
    """
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(
            tqdm(executor.map(download_bin, urls, filepathes), total=len(urls))
        )
        del results
        return


def csv_from_xls(xls_filepath, csv_filepath, sheet="Sheet1", enc="utf-8"):
    data_xls = pd.read_excel(xls_filepath, sheet, index_col=None)
    data_xls.to_csv(csv_filepath, encoding=enc)


def download_csv(url, filepath, enc="utf-8", dec="utf-8", logging=False):
    """
    Request a HTTP GET method to the given url (for REST API)
    and save its response as the csv file.

    Args:
    =====
    url: string
        valid url for REST API
    filepathe: string
        valid path to the destination file
    enc: string
        encoding type for a content in a given url
    dec: string
        decoding type for a content in a downloaded file
            dec = 'utf-8' for general env
            dec = 'sjis'  for Excel on Win
            dec = 'cp932' for Excel with extended JP str on Win
    logging: True/False
        flag whether putting process log
    """
    try:
        if logging:
            print("HTTP GET", url)
        r = requests.get(url, stream=True)
        with open(filepath, 'w', encoding=enc) as f:
            f.write(r.content.decode(dec))
    except requests.exceptions.RequestException as error:
        print(error)


def download_all_csv(
        urls,
        filepathes,
        max_workers=10,
        enc="utf-8",
        dec="utf-8"):
    """
    Request some HTTP GET methods to the given urls (for REST API)
    and save each response as the csv file.
    (!! This method uses multi threading when calling HTTP GET requests
    and downloading files in order to improve the processing speed.)

    Args:
    =====
    urls: list of strings
        valid urls for REST API
    filepathes: list of strings
        valid pathes to the destination file
    max_workers: int
        max number of working threads of CPUs within executing this method.
    enc: string
        encoding type for a content in a given url
    dec: string
        decoding type for a content in a downloaded file
            dec = 'utf-8' for general env
            dec = 'sjis'  for Excel on Win
            dec = 'cp932' for Excel with extended JP str on Win
    logging: True/False
    """
    func = functools.partial(download_csv, enc=enc, dec=dec)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(
            tqdm(executor.map(func, urls, filepathes), total=len(urls))
        )
        del results


def download_zip(url, filepath):
    try:
        r = requests.get(url, stream=True)
        with open(filepath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
                    f.flush()
    except requests.exceptions.RequestException as error:
        print(error)
        return False


def download_all_zip(urls, filepathes, max_workers=5):
    """
    Request some HTTP GET methods to the given urls (for REST API)
    and save each response as the local file.
    (!! This method uses multi threading when calling HTTP GET requests
    and downloading files in order to improve the processing speed.)

    Args:
    =====
    urls: list of strings
        valid urls for REST API
    filepathes: list of strings
        valid pathes to the destination files
    max_workers: int
        max number of working threads of CPUs within executing this method.
    """
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(
            tqdm(executor.map(download_zip, urls, filepathes), total=len(urls))
        )
        del results
        return


def extract_zip(filepath, target_dir):
    """
    Extract zip-file in the target directory.

    Args:
    =====
    filepath: string
        valid path to the target .zip file
    target_dir: path string
        valid dir path to the target directory where extracted files are saved
    """
    with zipfile.ZipFile(filepath) as f_zip:
        f_zip.extractall(target_dir)


def extract_all_zip(filepathes, target_dirs, max_workers=5):
    """
    Extract zip-file in the target directory.
    (!! This method uses multi threading when calling HTTP GET requests
    and downloading files in order to improve the processing speed.)

    Args:
    =====
    filepathes: list of strings
        valid pathes to the target .zip files
    target_dirs: list of path strings
        valid dir pathes to the target directories where extracted files are saved
    max_worker: int
        max number of working threads of CPUs within executing this method
    """
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(
            tqdm(executor.map(extract_zip, filepathes,
                              target_dirs), total=len(filepathes))
        )
        del results
        return

Main program  sample1

In [11]:
import json
from pprint import pprint
#from estat_api import EstatRestApiClient

"""
国勢調査(2015) > 人口等基本集計にある統計表のリストをJSONファイルで取得
"""

estat_api_client = EstatRestApiClient(app_id= "appid......")
json_dict = estat_api_client.getStatsList(lang="J", surveyYears="2015", statsCode="00200521", searchWord="人口等基本集計", limit=10, format="json")

pprint(json_dict)

filepath = "./example1.json"
json_str = json.dumps(json_dict, indent=2, ensure_ascii=False)
with open(filepath, "w") as f:
    f.write(json_str)

<Response [200]> HTTP GET: https://api.e-stat.go.jp/rest/3.0/app/json/getStatsList?lang=J&surveyYears=2015&statsCode=00200521&searchWord=%E4%BA%BA%E5%8F%A3%E7%AD%89%E5%9F%BA%E6%9C%AC%E9%9B%86%E8%A8%88&limit=10&appId=ea254e7aa27cdb7a49c649a2687a11e1628b7373
{'GET_STATS_LIST': {'DATALIST_INF': {'NUMBER': 139,
                                     'RESULT_INF': {'FROM_NUMBER': 1,
                                                    'NEXT_KEY': 11,
                                                    'TO_NUMBER': 10},
                                     'TABLE_INF': [{'@id': '0003148500',
                                                    'COLLECT_AREA': '該当なし',
                                                    'CYCLE': '-',
                                                    'DESCRIPTION': '',
                                                    'GOV_ORG': {'$': '総務省',
                                                                '@code': '00200'},
                                      

Main program  sample2

In [14]:
import csv
from tqdm import tqdm
#from estat_api import EstatRestApiClient

"""
国勢調査(2015)にある統計表のリストをCSVファイルで取得
"""
estat_api_client = EstatRestApiClient(app_id= "appid......")
json_dict = estat_api_client.getStatsList(lang="J", surveyYears="2015", statsCode="00200521", format="json")

pprint(json_dict)

filepath = "./example2.csv"
with open(filepath, 'w', encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["政府統計コード", "政府統計名", "集計名", "統計表ID(StatsDataId)", "統計表名", "表章地域"])

    table_list = json_dict["GET_STATS_LIST"]["DATALIST_INF"]["TABLE_INF"]
    for table in tqdm(table_list):
        tokei_code = table["STAT_NAME"]["@code"]

        if "TABULATION_CATEGORY" in table["STATISTICS_NAME_SPEC"]:
            tokei_name = table["STATISTICS_NAME_SPEC"]["TABULATION_CATEGORY"]
        else:
            tokei_name = ""

        if "TABULATION_SUB_CATEGORY1" in table["STATISTICS_NAME_SPEC"]:
            shukei_name = table["STATISTICS_NAME_SPEC"]["TABULATION_SUB_CATEGORY1"]
        else:
            shukei_name = ""

        if "TABLE_NAME" in table["TITLE_SPEC"]:
            table_name = table["TITLE_SPEC"]["TABLE_NAME"]
        else:
            table_name = ""

        if "TABLE_SUB_CATEGORY1" in table["TITLE_SPEC"]:
            table_areaunit = table["TITLE_SPEC"]["TABLE_SUB_CATEGORY1"]
        else:
            table_areaunit = ""

        table_id = table["@id"]

        row = [tokei_code, tokei_name, shukei_name, table_id, table_name, table_areaunit]
        writer.writerow(row)

ストリーミング出力は最後の 5000 行に切り捨てられました。
                                                                   '男女，年齢（5歳階級），労働力状態別人口（15歳以上）－全国，都道府県，市区町村',
                                                              '@no': '1-1'},
                                                    'TITLE_SPEC': {'TABLE_CATEGORY': '労働力状態・産業・職業・従業上の地位の不詳補完（就業状態等基本集計に対応）',
                                                                   'TABLE_NAME': '男女，年齢（5歳階級），労働力状態別人口（15歳以上）－全国，都道府県，市区町村'},
                                                    'UPDATED_DATE': '2024-09-30'},
                                                   {'@id': '0004019440',
                                                    'COLLECT_AREA': '市区町村',
                                                    'CYCLE': '-',
                                                    'DESCRIPTION': '',
                                                    'GOV_ORG': {'$': '総務省',
                                                                '@code': '00200'},


100%|██████████| 567/567 [00:00<00:00, 63309.83it/s]
